In [1]:
import pandas as pd
import numpy as np
import json
import datetime 

In [2]:
sourceCheckins = pd.read_csv('data/July/server/checkins.tsv', sep='\t')
checkins = pd.read_csv('data/July/server/checkinsHistory.tsv', sep='\t')
checkins1 = pd.read_csv('data/July/server/extendedCheckinsHistoryJune.tsv', sep='\t')
checkpoints = pd.read_csv('data/July/server/checkPoints.tsv', sep='\t')

In [17]:
len(np.unique(allCheckins.userId.tolist()))

707

In [12]:
len(allCheckins)

129741

In [4]:
allCheckins = allCheckins.drop_duplicates(subset = ['checkinId'])

In [3]:
allCheckins = pd.concat([checkins1, checkins])

In [3]:
allCheckins = pd.read_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t')

In [11]:
allCheckins.to_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t', encoding='utf-8')

In [13]:
allCheckins[.drop('Unnamed: 0.1', axis=1, inplace=True)]

,categoryId,categoryName,checkinId,checkinsCount,createdAt,createdAtDate,lat,lng,name,tipCount,userId,usersCount,venueId
0,4bf58dd8d48988d1fd931735,Metro Station,558ac442498e91c19449d814,14585,1435157570,2015-06-24 17:52:50,55.782920,37.720699,Метро Семёновская (metro Semyonovskaya),26,74434491,3460,4bed6c2c9868a5933c555d46
1,4bf58dd8d48988d1d3941735,Vegetarian / Vegan Restaurant,558ababc498e7aa1a83dfb16,10096,1435155132,2015-06-24 17:12:12,55.758369,37.657153,Джаганнат,222,74434491,4972,4d8f05153bfef04d84a6a586


In [17]:
sortedVenues = []
sortedVenuesIds = []
for venueId, group in allCheckins.groupby('venueId').agg('size').order(ascending = False).iteritems():
    sortedVenues.append({'venueId':venueId,
                        'group':group})
    sortedVenuesIds.append(venueId)

In [115]:
#'World Class Lite in top ten!, 520f117111d2a782f1211c3b'

In [15]:
groupedUsers = allCheckins.groupby('userId')
userVenues = []
for index, group in enumerate(groupedUsers):
        userVenues.append(group[1]['venueId'].tolist())

In [16]:
len(userVenues)

1142

In [18]:
top50VenuesCheckins = allCheckins[allCheckins.venueId.isin(sortedVenuesIds[:50])]

In [23]:
for venueId in sortedVenuesIds[:50]:
    print venueNameFromId(venueId)

ЦПКиО им. Горького / Gorky Park
Москва / Moscow
Парк искусств «Музеон» / Muzeon Park
ВДНХ (Выставка достижений народного хозяйства)
Россия / Russia
Международный аэропорт Домодедово / Domodedovo International Airport (DME)
ПКиО «Сокольники»
Красная площадь / Red Square
ТРЦ «Европейский»
Арбат / Arbat Street
Международный аэропорт Шереметьево / Sheremetyevo International Airport (SVO)
ТРК «Атриум» / Atrium Mall
ТЦ «Метрополис»
Афимолл Сити / Afimall City
Физика
Центральный детский магазин
Международный аэропорт Внуково / Vnukovo International Airport (VKO)
Ленинградский вокзал / Leningradsky Railway Terminal
Музей-заповедник «Коломенское»
Нескучный сад
ТЦ «Охотный ряд»
ТРЦ «Авиапарк»
Планета Фитнес
World Class Lite
ГУМ / GUM
Воробьёвская набережная
Патриаршие пруды / Patriarshiye Ponds
ПКиО «Измайловский»
Московский вокзал / Moskovsky Railway Station
Tsvetnoy Central Market
Парк Победы (Поклонная гора) / Victory Park
Фитнес-мания
Казанский вокзал / Kazansky Rail Terminal
ТРЦ «Золотой Ва

In [ ]:
from pymining import itemmining
relim_input = itemmining.get_relim_input(userVenues)
report = itemmining.relim(relim_input, min_support=2)
report

In [24]:
def load_dataset():
    "Load the sample dataset."
    return userVenues
 
 
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a ket of a dictionary.
    return map(frozenset, c1)
 
 
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1
 
    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data
 
 
def aprioriGen(freq_sets, k):
    "Generate the joint transactions from candidate sets"
    retList = []
    lenLk = len(freq_sets)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(freq_sets[i])[:k - 2]
            L2 = list(freq_sets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(freq_sets[i] | freq_sets[j])
    return retList
 
 
def apriori(dataset, minsupport=0.5):
    "Generate a list of candidate item sets"
    C1 = createC1(dataset)
    D = map(set, dataset)
    L1, support_data = scanD(D, C1, minsupport)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        support_data.update(supK)
        L.append(Lk)
        k += 1
 
    return L, support_data

In [251]:
C1 = createC1(userVenues)

In [252]:
D = map(set, userVenues)

In [253]:
L1, support_data = scanD(D, C1, 0.01)

In [254]:
L, supportData = apriori(userVenues, minsupport=0.01)

In [255]:
frequentVenues = []
for itemset, support in zip(L, supportData):
    if len(itemset)>5:
        print support
        frequentVenueSet = []
        for item in itemset:
            frequentVenueSet.append(next(enumerate(item))[1])
        frequentVenues.append(frequentVenueSet)

frozenset(['4be6909fd4f7c9b6b2e02620', '4ba6146ef964a5202d3239e3'])
frozenset(['4bdd5f27f219c9b6bb751210', '4c77623f59a3236a4abec118', '4c0772808a81c9b687392790'])
frozenset(['4d01c5cd68e38eec0fb2dec4', '4c98db097b9d37047ab6e185'])
frozenset(['55604879498e39378e6294e8', '4b9cf21af964a5200e8436e3'])
frozenset(['4fca7a60e4b0d8a39128edcb'])
frozenset(['4b645287f964a52095aa2ae3', '4bdd5f27f219c9b6bb751210', '4bdc9716afe8c9b689235085', '4b76c7bdf964a520a85e2ee3'])


In [40]:
len(userVenues)*0.0015455950541

1.0000000000027

In [256]:
for venueId in frequentVenues[0]:
    print allCheckins[allCheckins.venueId == venueId][:1]['name']

630    Афимолл Сити / Afimall City
Name: name, dtype: object
27878    Парк у Яузы
Name: name, dtype: object
5736    Гин-но Таки
Name: name, dtype: object
2236    McDonald's
Name: name, dtype: object
5207    Hard Rock Cafe Moscow
Name: name, dtype: object
1273    Bar BQ Cafe
Name: name, dtype: object
6545    Страна которой нет
Name: name, dtype: object
4678    Каро Vegas 22
Name: name, dtype: object
9716    ДжонДжоли
Name: name, dtype: object
9002    Счастье
Name: name, dtype: object
11650    Cuba Libre
Name: name, dtype: object
2998    Площадь Восстания
Name: name, dtype: object
1526    Starbucks
Name: name, dtype: object
1659    Аэроэкспресс - Терминал на Павелецком вокзале
Name: name, dtype: object
853    ТРK «Вегас» (Крокус Сити)
Name: name, dtype: object
7508    Biblioteka
Name: name, dtype: object
4082    Декатлон
Name: name, dtype: object
2190    Теплый Стан
Name: name, dtype: object
2445    Площадь Революции
Name: name, dtype: object
2332    Государственная Третьяковская галерея

In [257]:
def generateRules(L, support_data, min_confidence=0.7):
    """Create the association rules
    L: list of frequent item sets
    support_data: support data for those itemsets
    min_confidence: minimum confidence threshold
    """
    rules = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            #print "freqSet", freqSet, 'H1', H1
            if (i > 1):
                rules_from_conseq(freqSet, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freqSet, H1, support_data, rules, min_confidence)
    return rules
 
 
def calc_confidence(freqSet, H, support_data, rules, min_confidence=0.7):
    "Evaluate the rule generated"
    pruned_H = []
    for conseq in H:
        conf = support_data[freqSet] / support_data[freqSet - conseq]
        if conf >= min_confidence:
            print freqSet - conseq, '--->', conseq, 'conf:', conf
            rules.append((freqSet - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H
 
 
def rules_from_conseq(freqSet, H, support_data, rules, min_confidence=0.7):
    "Generate a set of candidate rules"
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calc_confidence(freqSet, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            rules_from_conseq(freqSet, Hmp1, support_data, rules, min_confidence)

In [261]:
rules = generateRules(L, supportData, min_confidence=0.6)

frozenset(['4bdc2c0d2a3a0f4733b5b1b6']) ---> frozenset(['4bdd5f27f219c9b6bb751210']) conf: 0.6875
frozenset(['4d7b42a3f260a093113124ba']) ---> frozenset(['4bdd5f27f219c9b6bb751210']) conf: 0.644295302013
frozenset(['4c4ac66b42b4d13a087bb57d']) ---> frozenset(['4bdd5f27f219c9b6bb751210']) conf: 0.647058823529
frozenset(['4babcd62f964a520d9ca3ae3']) ---> frozenset(['4bdd5f27f219c9b6bb751210']) conf: 0.636363636364
frozenset(['4bad3d17f964a520ce3c3be3']) ---> frozenset(['4bdd5f27f219c9b6bb751210']) conf: 0.69387755102
frozenset(['4d7a08c6b359f04d3e62d014']) ---> frozenset(['4e4aab0f62e13240b700e034']) conf: 0.705882352941
frozenset(['4c6d4302f2846dcba80a65e9']) ---> frozenset(['4bdd5f27f219c9b6bb751210']) conf: 0.75
frozenset(['4e6a3fa5fa76cd0f769ca400']) ---> frozenset(['4bc090ca920eb7132e14192c']) conf: 0.666666666667
frozenset(['4c088729009a0f478a58e6bf']) ---> frozenset(['4bdd5f27f219c9b6bb751210']) conf: 0.736842105263
frozenset(['540745a2498e1d19d596d818']) ---> frozenset(['4bdd5f27

In [20]:
venuesDict = {}
def venueNameFromId(venueId):
    if venueId not in venuesDict.keys():
        venuesDict[venueId] = allCheckins[allCheckins.venueId == venueId].iloc[0]['name']
    return venuesDict[venueId]

In [21]:
venuesCatDict = {}
def venueCategoryFromId(venueId):
    if venueId not in venuesDict.keys():
        venuesDict[venueId] = allCheckins[allCheckins.venueId == venueId].iloc[0]['categoryName']
    return venuesDict[venueId]

In [22]:
categoriesDict = {}
def categoryNameFromId(categoryId):
    if categoryId not in categoriesDict.keys():
        categoriesDict[categoryId] = allCheckins[allCheckins.categoryId == categoryId].iloc[0]['categoryName']
    return categoriesDict[categoryId]

In [45]:
def countUsersAtVenue(venuedId):
    return len(np.unique(allCheckins[allCheckins.venueId == venueId]['userId'].tolist()))

In [262]:
for rule in rules:
    simpleRule = 1
    if len(rule[0]) < 2:
        continue
        
    for venueId in enumerate(rule[0]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    simpleRule = 1
    
    for venueId in enumerate(rule[1]):
        if venueId[1] not in sortedVenuesIds[:20]:
            simpleRule = 0
            
    if simpleRule:
        continue
    
    for venueId in enumerate(rule[0]):
        print venueNameFromId(venueId[1])
        
    print '--->'
    for venueId in enumerate(rule[1]):
        print venueNameFromId(venueId[1])
    print 'conf:', rule[2]
    print ''

Афимолл Сити / Afimall City
Московский вокзал / Moskovsky Railway Station
--->
ЦПКиО им. Горького / Gorky Park
Ленинградский вокзал / Leningradsky Railway Terminal
conf: 0.6

ВДНХ (Выставка достижений народного хозяйства)
Московский вокзал / Moskovsky Railway Station
--->
ЦПКиО им. Горького / Gorky Park
Ленинградский вокзал / Leningradsky Railway Terminal
conf: 0.645161290323

ТРЦ «Галерея» / Galeria Shopping Mall
Московский вокзал / Moskovsky Railway Station
--->
Ленинградский вокзал / Leningradsky Railway Terminal
ВДНХ (Выставка достижений народного хозяйства)
conf: 0.666666666667

ТРЦ «Галерея» / Galeria Shopping Mall
ВДНХ (Выставка достижений народного хозяйства)
--->
Ленинградский вокзал / Leningradsky Railway Terminal
Московский вокзал / Moskovsky Railway Station
conf: 0.695652173913

Парк искусств «Музеон» / Muzeon Park
ТРЦ «Галерея» / Galeria Shopping Mall
--->
ЦПКиО им. Горького / Gorky Park
Ленинградский вокзал / Leningradsky Railway Terminal
conf: 0.733333333333

Невский про

In [45]:
supportData[frozenset(frequentVenues[1])]

KeyError: frozenset(['4bfc9df283bbd13aa4d462c5', '4bdd5f27f219c9b6bb751210', '4bb3345942959c74d79d212c', '4bc090ca920eb7132e14192c'])

In [ ]:
#Using code for rules from here: http://aimotion.blogspot.ru/2013/01/machine-learning-and-data-mining.html

In [268]:
multiUserSortedVenues = filter(lambda x: x['group'] > 1, sortedVenues)

In [25]:
userVenueGroups = []
for venueId, group in allCheckins.groupby('venueId'):
    if len(group.groupby('userId'))>1:
        userVenueGroups.append(venueId)

In [26]:
userVenueCategories = []
for categoryId, group in allCheckins.groupby('categoryId'):
    if len(group.groupby('userId'))>1:
        userVenueCategories.append(categoryId)

In [27]:
#Make lists of users check-in venues
multiUserCheckins = allCheckins[allCheckins.venueId.isin(userVenueGroups)]
groupedUsers = multiUserCheckins.groupby('userId')
userVenuesList = []
userIds = []
for index, group in enumerate(groupedUsers):
        userVenuesList.append(group[1]['venueId'].tolist())
        userIds.append(group[0])

In [28]:
#Make lists of users check-in catrgories
fineCheckins = allCheckins[allCheckins.categoryId.isin(userVenueCategories)]
groupedUsers = fineCheckins.groupby('userId')
userCategoryList = []
userIdsForCat = []
for index, group in enumerate(groupedUsers):
        userCategoryList.append(group[1]['categoryId'].tolist())
        userIdsForCat.append(group[0])

# K-means, spectral

In [29]:
#Let's build some TF-ID vectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.cluster import MiniBatchKMeans

In [30]:
venuesByUser = []
for userList in userVenuesList:
    venuesByUser.append(' '.join(userList))
    
categoriesByUser = []
for userList in userCategoryList:
    try:
        categoriesByUser.append(' '.join(userList))
    except:
        print userList

tfidf_vectorizer = TfidfVectorizer(vocabulary = userVenueGroups)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(venuesByUser)

In [31]:
from time import time
cocluster = SpectralCoclustering(n_clusters=20,
                                 svd_method='arpack', random_state=0)
kmeans = MiniBatchKMeans(n_clusters=20,
                         random_state=0)

print("Coclustering...")
start_time = time()
cocluster.fit(tfidf_matrix_train)
y_cocluster = cocluster.row_labels_
print("Done in {:.2f}s.".format(
    time() - start_time))

print("MiniBatchKMeans...")
start_time = time()
y_kmeans = kmeans.fit_predict(tfidf_matrix_train)
print("Done in {:.2f}s.".format(
    time() - start_time))

feature_names = tfidf_vectorizer.get_feature_names()

Coclustering...
Done in 0.50s.
MiniBatchKMeans...
Done in 0.14s.


In [32]:
clusters = x = [[] for i in range(20)]
for index, clustedId in enumerate(y_kmeans):
    clusters[clustedId].append(userIds[index])

# LDA

In [33]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 

cv = sklearn.feature_extraction.text.CountVectorizer()
mat = cv.fit_transform(venuesByUser).toarray()
print('Done Document matrix')

Done Document matrix


In [34]:
cvCategory = sklearn.feature_extraction.text.CountVectorizer()
matCategory = cvCategory.fit_transform(categoriesByUser).toarray()
print('Done Document matrix')

Done Document matrix


In [35]:
len(userVenueCategories)

516

In [36]:
venuesVocab = cv.get_feature_names()
print len(venuesVocab)

8282


In [37]:
categoriesVocab = cvCategory.get_feature_names()
print len(categoriesVocab)

516


In [51]:
import lda
model20 = lda.LDA(n_topics=20, n_iter=2000, random_state=1,alpha = 0.001)
model20.fit(mat)  # model.fit_transform(X) is also available
topic_word = model20.topic_word_  # model.components_ also works
print('Done LDA fitting')

Done LDA fitting


In [39]:
import lda
model20Cat = lda.LDA(n_topics=20, n_iter=2000, random_state=1,alpha = 0.001)
model20Cat.fit(matCategory)  # model.fit_transform(X) is also available
topic_cats = model20Cat.topic_word_  # model.components_ also works
print('Done LDA fitting')

Done LDA fitting


In [52]:
n_top_venues = 10
for i, venues_dist in enumerate(topic_word):
    topic_venues = np.array(venuesVocab)[np.argsort(venues_dist)][:-n_top_venues:-1]
    print('Cluster {}:'.format(i))
    for venueId in topic_venues:
        print("{}, {} users ").format(venueNameFromId(venueId),countUsersAtVenue(venueId))
    print ''

Cluster 0:
Фитнес клуб "Зебра"/ ZEBRA FITNES and SPA, 7 users 
МГЛУ / Московский государственный лингвистический университет / Moscow State Linguistic University, 11 users 
moonrise kingdom 🌜, 2 users 
World Class, 2 users 
Государственный университет управления (ГУУ), 9 users 
РАНХиГС при Президенте РФ, 19 users 
Минобороны России, 2 users 
ТРЦ «Европейский», 185 users 
Бизнес-центр Экоофис, 2 users 

Cluster 1:
Фитнес-мания, 15 users 
Международный аэропорт Домодедово / Domodedovo International Airport (DME), 261 users 
Международный аэропорт Шереметьево / Sheremetyevo International Airport (SVO), 178 users 
Международный аэропорт Внуково / Vnukovo International Airport (VKO), 154 users 
Alex Fitness, 2 users 
Терминал D / Terminal D, 55 users 
Coca-Cola HBC Eurasia, 2 users 
Кампус, 10 users 
Битцевский лесопарк, 21 users 

Cluster 2:
Москва / Moscow, 229 users 
Россия / Russia, 115 users 
Западный административный округ, 51 users 
ЦПКиО им. Горького / Gorky Park, 599 users 
Красная

In [59]:
len(model20.ndz_)

1142

In [41]:
n_top_venues = 10
for i, venues_dist in enumerate(topic_cats):
    topic_venues = np.array(categoriesVocab)[np.argsort(venues_dist)][:-n_top_venues:-1]
    print('Cluster {}:'.format(i))
    for categoryId in topic_venues:
        #print categoryId
        print(categoryNameFromId(categoryId))
    print ''

Cluster 0:
Office
Residential Building (Apartment / Condo)
Coworking Space
Mall
Government Building
Gas Station / Garage
Assisted Living
Building
Campaign Office

Cluster 1:
University
Mall
Park
Coffee Shop
Residential Building (Apartment / Condo)
Plaza
College Academic Building
Street
Home (private)

Cluster 2:
Gym / Fitness Center
Park
Pool
Building
Government Building
Medical Center
Airport Terminal
Stadium
Multiplex

Cluster 3:
Office
Mall
Park
Home (private)
Train Station
Coffee Shop
Sushi Restaurant
Medical Center
Bank

Cluster 4:
Residential Building (Apartment / Condo)
Park
Grocery Store
Supermarket
Street
Neighborhood
Lake
Platform
Plaza

Cluster 5:
Gym
College Lab
Sushi Restaurant
Public Art
Medical Center
Moving Target
Caucasian Restaurant
Salon / Barbershop
Photography Studio

Cluster 6:
Dance Studio
College Residence Hall
Government Building
Housing Development
Gym Pool
Building
Train Station
College Academic Building
Garden

Cluster 7:
Metro Station
Train Station
Bus Stop

In [228]:
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).intersection(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))
print len(set(multiUserCheckins[multiUserCheckins.userId == clusters[10][1]]['venueId'].tolist()).union(multiUserCheckins[multiUserCheckins.userId == clusters[10][2]]['venueId'].tolist()))

4
140


In [ ]:

Cluster 0:
park
mall
shop
coffee
college
building
university
plaza
arts

Cluster 1:
college
building
university
condo
residential
apartment
academic
hall
residence

Cluster 2:
private
home
station
mall
food
fast
gas
plaza
restaurant

Cluster 3:
mall
store
park
shop
restaurant
food
street
station
coffee

Cluster 4:
university
school
mall
park
middle
college
home
private
center

Cluster 5:
building
residential
condo
apartment
store
park
mall
supermarket
grocery

Cluster 6:
city
bar
restaurant
park
country
nightclub
café
neighborhood
plaza

Cluster 7:
station
metro
train
bus
stop
home
mall
platform
private

Cluster 8:
restaurant
park
mall
shop
pool
airport
school
coffee
café

Cluster 9:
space
coworking
studio
dance
event
building
cultural
government
garden

Cluster 10:
home
private
building
store
street
park
mall
neighborhood
residential

Cluster 11:
building
city
government
store
station
office
mall
park
garage

Cluster 12:
office
park
tech
mall
startup
dentist
train
station
building

Cluster 13:
center
gym
fitness
medical
office
salon
park
barbershop
restaurant

Cluster 14:
restaurant
park
bar
airport
shop
café
hotel
art
coffee

Cluster 15:
gym
center
fitness
mall
airport
restaurant
park
bar
hotel

Cluster 16:
park
street
plaza
neighborhood
mall
store
other
shop
outdoors

Cluster 17:
restaurant
airport
hotel
café
eastern
sushi
factory
bar
city

Cluster 18:
housing
development
general
entertainment
soccer
stadium
center
non
profit

Cluster 19:
shop
restaurant
coffee
salon
mall
café
bar
eastern
barbershop
